In [1]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [11]:



# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun PERSON
Recode ORG
earlier this week DATE


In [70]:
import fitz

doc = fitz.open("Data/timemach.pdf") 
page = doc[4]

def highlight_word(page, word):
  quads = page.search_for(word)
  
  for quad in quads:
    annot = page.add_highlight_annot(quad)
    annot.set_colors({"stroke": (1, 1, 0)})
    annot.update()

word = 'Traveller'

highlight_word(page, " " + word + " " )

doc.save("highlighted.pdf")
doc.close()